In [88]:
from keras.layers import Dense, LeakyReLU, Dropout
from keras import regularizers
from keras.models import Model, Sequential
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from contrastive import CPCA
import matplotlib.pyplot as plt

In [89]:
X = pd.read_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/all_genes_norm.csv", index_col = 0)
X.head()

,E001h3k4me3,E002h3k4me3,E003h3k4me3,E004h3k4me3,E005h3k4me3,E006h3k4me3,E007h3k4me3,E008h3k4me3,E009h3k4me3,E010h3k4me3,...,E056tensor,E059tensor,E089tensor,E090tensor,E094tensor,E097tensor,E098tensor,E100tensor,E109tensor,epit.entropy
ENSG00000004866,0.296625,0.105981,0.249337,0.184611,0.007758,-0.103241,-0.254451,-0.153236,0.567978,2.079856,...,0.525170,1.154039,0.094809,0.718738,0.699267,0.061643,1.580294,2.929246,0.954994,0.658541
ENSG00000005339,0.664484,0.469639,0.848853,1.325303,0.711186,0.610536,-0.522232,0.906287,0.822634,1.136299,...,-0.499322,0.003369,0.400658,-0.011319,0.565993,0.335474,0.384709,-0.146969,-0.043804,0.821583
ENSG00000005513,0.204398,1.045012,0.147311,-0.064777,1.585373,2.315699,-0.443473,0.536371,1.998647,0.870081,...,0.411337,0.642630,0.706508,0.171195,0.832542,0.198559,-0.158738,-0.293455,-0.614546,0.555202
ENSG00000006704,-0.035464,0.227201,0.362198,0.396463,1.144966,1.109917,1.588511,1.286247,0.893187,0.437056,...,0.069840,-0.252335,0.298708,0.079938,-0.233652,-0.075272,-0.267428,-0.439942,-0.186490,1.063549
ENSG00000007168,0.896981,0.367413,0.612749,1.201246,0.086256,0.387711,0.454905,0.145055,1.162038,1.392407,...,0.752834,0.898335,0.910407,0.536223,0.965816,1.156964,0.710778,0.878436,0.812308,0.392160


In [90]:
X.shape

(19430, 359)

In [91]:
X_sample = pd.read_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/2000_sample_norm.csv", index_col = 0)
X_sample.head()

,E001h3k4me3,E002h3k4me3,E003h3k4me3,E004h3k4me3,E005h3k4me3,E006h3k4me3,E007h3k4me3,E008h3k4me3,E009h3k4me3,E010h3k4me3,...,E056tensor,E059tensor,E089tensor,E090tensor,E094tensor,E097tensor,E098tensor,E100tensor,E109tensor,epit.entropy
ENSG00000115221,-1.203563,-0.136458,-1.214242,-1.136124,0.233568,-1.109903,-0.954356,-1.228917,-1.171202,-1.145929,...,-0.840819,-0.891597,-0.822740,-0.923890,-0.900022,-0.896762,-0.919565,-0.879401,-0.899917,-1.477453
ENSG00000117410,-0.357033,-0.271642,0.447520,0.082184,0.473651,0.164360,0.711135,4.023710,2.602726,-0.027983,...,1.435829,2.049005,2.235755,2.270108,1.632186,1.704624,1.471604,2.343300,1.525735,0.693065
ENSG00000249115,-0.560074,-0.362138,-0.536626,-0.442038,1.829024,-0.386118,0.250657,0.117977,-0.010636,0.048260,...,-0.840819,-0.891597,-0.822740,-0.923890,-0.900022,-0.896762,-0.919565,-0.879401,-0.899917,-1.477453
ENSG00000063761,-0.683512,-0.464364,-0.288332,0.334752,-0.646736,-0.493579,-0.485478,-0.522278,-0.466930,-0.461429,...,0.297505,0.898335,0.298708,0.536223,1.099090,0.883134,1.145536,1.171409,0.812308,0.688392
ENSG00000135047,-0.291457,0.270773,-0.362369,-0.095950,-0.109480,0.154878,0.015430,-0.356318,0.331272,-0.168252,...,0.297505,1.281892,0.298708,0.353709,0.699267,1.430794,-0.919565,-0.000483,-0.186490,0.826255


In [92]:
X_sample.shape

(2000, 359)

In [93]:
Model1 = Sequential()
Model1.add(Dense(100,input_shape = (359,),name = 'inter1'))
Model1.add(LeakyReLU(alpha=0.3))
Model1.add(Dense(359,))
Model1.add(LeakyReLU(alpha=0.3))

Model1.compile(optimizer = 'adam',loss = 'mean_squared_error')
Model1.fit(X.values, X.values,
           epochs = 100, 
           batch_size = 64,
           validation_data = (X_sample.values,X_sample.values))

Train on 19430 samples, validate on 2000 samples
Epoch 1/100
19430/19430 [==============================] - 2s 90us/step - loss: 0.4075 - val_loss: 0.2843
Epoch 2/100
19430/19430 [==============================] - 1s 69us/step - loss: 0.2477 - val_loss: 0.2224
Epoch 3/100
19430/19430 [==============================] - 1s 69us/step - loss: 0.2055 - val_loss: 0.1957
Epoch 4/100
19430/19430 [==============================] - 1s 75us/step - loss: 0.1872 - val_loss: 0.1842
Epoch 5/100
19430/19430 [==============================] - 1s 68us/step - loss: 0.1762 - val_loss: 0.1738
Epoch 6/100
19430/19430 [==============================] - 1s 67us/step - loss: 0.1704 - val_loss: 0.1678
Epoch 7/100
19430/19430 [==============================] - 1s 67us/step - loss: 0.1650 - val_loss: 0.1623
Epoch 8/100
19430/19430 [==============================] - 1s 68us/step - loss: 0.1618 - val_loss: 0.1614
Epoch 9/100
19430/19430 [==============================] - 1s 68us/step - loss: 0.1609 - val_loss: 0.15

19430/19430 [==============================] - 1s 67us/step - loss: 0.1339 - val_loss: 0.1334
Epoch 78/100
19430/19430 [==============================] - 1s 66us/step - loss: 0.1334 - val_loss: 0.1334
Epoch 79/100
19430/19430 [==============================] - 1s 66us/step - loss: 0.1334 - val_loss: 0.1323
Epoch 80/100
19430/19430 [==============================] - 1s 74us/step - loss: 0.1330 - val_loss: 0.1336
Epoch 81/100
19430/19430 [==============================] - 2s 86us/step - loss: 0.1337 - val_loss: 0.1348
Epoch 82/100
19430/19430 [==============================] - 2s 86us/step - loss: 0.1339 - val_loss: 0.1333
Epoch 83/100
19430/19430 [==============================] - 2s 100us/step - loss: 0.1329 - val_loss: 0.1334
Epoch 84/100
19430/19430 [==============================] - 2s 77us/step - loss: 0.1329 - val_loss: 0.1316
Epoch 85/100
19430/19430 [==============================] - 1s 73us/step - loss: 0.1330 - val_loss: 0.1343
Epoch 86/100
19430/19430 [=======================

In [94]:
middle1 = Model(inputs = Model1.input,outputs = Model1.get_layer('inter1').output)
middle1_out = middle1.predict(X.values)
middle1_out_vld = middle1.predict(X_sample.values)

In [95]:
np.max(middle1_out, 0)

array([ 10.31379032,   6.85312843,   5.65701246,   8.59158421,
        10.3438158 ,  12.9912653 ,   8.90900421,   6.21373749,
         6.20271063,  10.06014633,   8.68266487,   7.02023792,
        17.29282379,  11.46195316,   4.85800457,   6.74078369,
        11.59032631,   9.68308449,   5.98702621,   6.825037  ,
         6.18343496,  18.3867321 ,   6.38755894,   4.61319256,
        10.99646854,   4.64433479,   9.92975426,   5.2706399 ,
         8.80031776,  13.14039421,  10.10639858,   6.12761402,
        10.51253986,   9.55984783,  10.3263607 ,  10.20404625,
        12.51140976,  10.01408482,   7.32546425,   5.14242268,
         8.38302231,   5.9806695 ,  11.33290958,  11.40169907,
        12.86963177,   8.60473537,  10.59547806,  12.3857069 ,
         3.00383997,   6.82080126,  10.66193676,   3.81934476,
         4.05250835,   8.48815632,   7.71562099,   5.83293343,
         6.56346464,  15.45814991,  19.89453888,   6.45917845,
         5.56066561,   8.0441227 ,   8.47682667,   5.67

In [96]:
Model2 = Sequential()
Model2.add(Dense(50,input_shape = (100,),name = 'inter2'))
Model2.add(LeakyReLU(alpha = 0.3))
Model2.add(Dense(100,))
Model2.add(LeakyReLU(alpha = 0.3))
Model2.compile(optimizer = 'adam',loss = 'mean_squared_error')
Model2.fit(middle1_out,middle1_out,epochs = 100,batch_size = 64, validation_data = (middle1_out_vld,middle1_out_vld))          

Train on 19430 samples, validate on 2000 samples
Epoch 1/100
19430/19430 [==============================] - 2s 80us/step - loss: 3.2535 - val_loss: 2.2494
Epoch 2/100
19430/19430 [==============================] - 1s 30us/step - loss: 1.8474 - val_loss: 1.6063
Epoch 3/100
19430/19430 [==============================] - 1s 31us/step - loss: 1.4261 - val_loss: 1.3159
Epoch 4/100
19430/19430 [==============================] - ETA: 0s - loss: 1.215 - 1s 29us/step - loss: 1.2117 - val_loss: 1.1435
Epoch 5/100
19430/19430 [==============================] - 1s 27us/step - loss: 1.0774 - val_loss: 1.0286
Epoch 6/100
19430/19430 [==============================] - 1s 27us/step - loss: 0.9887 - val_loss: 0.9639
Epoch 7/100
19430/19430 [==============================] - 1s 29us/step - loss: 0.9291 - val_loss: 0.9089
Epoch 8/100
19430/19430 [==============================] - 1s 29us/step - loss: 0.8872 - val_loss: 0.8654
Epoch 9/100
19430/19430 [==============================] - 1s 28us/step - loss:

Epoch 77/100
19430/19430 [==============================] - 1s 32us/step - loss: 0.6456 - val_loss: 0.6469
Epoch 78/100
19430/19430 [==============================] - 1s 28us/step - loss: 0.6448 - val_loss: 0.6461
Epoch 79/100
19430/19430 [==============================] - 1s 28us/step - loss: 0.6441 - val_loss: 0.6426
Epoch 80/100
19430/19430 [==============================] - 1s 26us/step - loss: 0.6434 - val_loss: 0.6475
Epoch 81/100
19430/19430 [==============================] - 1s 26us/step - loss: 0.6422 - val_loss: 0.6444
Epoch 82/100
19430/19430 [==============================] - 1s 27us/step - loss: 0.6422 - val_loss: 0.6405
Epoch 83/100
19430/19430 [==============================] - 1s 27us/step - loss: 0.6413 - val_loss: 0.6454
Epoch 84/100
19430/19430 [==============================] - 1s 27us/step - loss: 0.6400 - val_loss: 0.6387
Epoch 85/100
19430/19430 [==============================] - 1s 27us/step - loss: 0.6393 - val_loss: 0.6423
Epoch 86/100
19430/19430 [===========

In [97]:
middle2 = Model(inputs = Model2.input, outputs = Model2.get_layer('inter2').output)
middle2_out = middle2.predict(middle1_out)
middle2_out_vld = middle2.predict(middle1_out_vld)

In [98]:
np.max(middle2_out, 0)

array([  76.06531525,   49.27150345,   52.35367203,  141.67564392,
         69.30776215,   85.77253723,   98.19774628,   57.50793457,
         37.42586899,  118.96064758,  129.3445282 ,   42.38468933,
        128.64471436,   76.76872253,   71.05738831,   50.81737518,
         58.40584946,  108.48958588,   50.15361404,  137.61569214,
         48.7522049 ,   52.7849884 ,   94.38228607,   61.93455505,
         50.61655426,   29.04439926,   56.17163849,   51.87415314,
        106.27674103,  128.70887756,   57.56659698,  112.82585144,
         84.95211792,   68.45430756,   66.93554688,   83.50508881,
         10.15779209,   61.95483017,   41.12686157,   91.71226501,
         56.93369675,   41.01479721,   50.22705078,   98.9598465 ,
        123.56528473,   63.40103149,   58.94500351,   41.58146667,
         88.21643066,   81.37075806], dtype=float32)

In [99]:
Model3 = Sequential()
Model3.add(Dense(20,input_shape = (50,),name = 'inter3'))
Model3.add(LeakyReLU(alpha = 0.3))
Model3.add(Dense(50,))
Model3.add(LeakyReLU(alpha = 0.3))
Model3.compile(optimizer = 'adam',loss = 'mean_squared_error')
Model3.fit(middle2_out,middle2_out,epochs = 100,batch_size = 64, validation_data = (middle2_out_vld,middle2_out_vld))

Train on 19430 samples, validate on 2000 samples
Epoch 1/100
19430/19430 [==============================] - 1s 38us/step - loss: 14.3715 - val_loss: 8.7826
Epoch 2/100
19430/19430 [==============================] - 0s 22us/step - loss: 7.2102 - val_loss: 6.3036
Epoch 3/100
19430/19430 [==============================] - 1s 27us/step - loss: 5.6342 - val_loss: 5.2209
Epoch 4/100
19430/19430 [==============================] - 0s 24us/step - loss: 4.8304 - val_loss: 4.5725
Epoch 5/100
19430/19430 [==============================] - 1s 27us/step - loss: 4.3218 - val_loss: 4.1759
Epoch 6/100
19430/19430 [==============================] - 0s 25us/step - loss: 3.9586 - val_loss: 3.8464
Epoch 7/100
19430/19430 [==============================] - 0s 24us/step - loss: 3.7076 - val_loss: 3.6310
Epoch 8/100
19430/19430 [==============================] - 0s 22us/step - loss: 3.5137 - val_loss: 3.4544
Epoch 9/100
19430/19430 [==============================] - 0s 22us/step - loss: 3.3791 - val_loss: 3.3

19430/19430 [==============================] - 0s 23us/step - loss: 2.7229 - val_loss: 2.7010
Epoch 78/100
19430/19430 [==============================] - 0s 22us/step - loss: 2.7208 - val_loss: 2.7249
Epoch 79/100
19430/19430 [==============================] - 0s 21us/step - loss: 2.7218 - val_loss: 2.7042
Epoch 80/100
19430/19430 [==============================] - 0s 22us/step - loss: 2.7147 - val_loss: 2.6909
Epoch 81/100
19430/19430 [==============================] - 0s 22us/step - loss: 2.7184 - val_loss: 2.7058
Epoch 82/100
19430/19430 [==============================] - 0s 21us/step - loss: 2.7163 - val_loss: 2.6824
Epoch 83/100
19430/19430 [==============================] - 1s 26us/step - loss: 2.7152 - val_loss: 2.6868
Epoch 84/100
19430/19430 [==============================] - 0s 24us/step - loss: 2.7172 - val_loss: 2.6864
Epoch 85/100
19430/19430 [==============================] - 1s 30us/step - loss: 2.7158 - val_loss: 2.6949
Epoch 86/100
19430/19430 [========================

In [100]:
middle3 = Model(inputs = Model3.input, outputs = Model3.get_layer('inter3').output)
middle3_out = middle3.predict(middle2_out)
middle3_out_vld = middle3.predict(middle2_out_vld)

In [101]:
np.max(middle3_out,0)

array([ 135.59967041,   65.82987213,   56.62382126,   30.11065292,
         28.74898148,   18.76107788,  106.70642853,   96.98205566,
         24.20425034,  127.88632965,   21.56210136,  146.52131653,
         18.7825985 ,   91.41913605,   20.55720711,   36.16321564,
         14.81036282,   12.38606644,   24.21364784,   14.81716919], dtype=float32)

In [102]:
df = pd.DataFrame(middle3_out)

In [103]:
df.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
19425,15.943581,9.866013,21.122356,-1.156925,-15.380181,-12.714820,4.594036,8.524107,-20.542030,9.037570,-15.814054,10.869353,-27.148956,15.287834,-11.020136,1.348209,-13.199464,-18.299294,-11.007733,-28.701838
19426,7.971561,11.447915,17.890844,-23.836309,-52.130943,-28.175690,11.610044,8.078794,-33.595161,12.370294,-45.403641,13.712380,-35.026283,12.411966,-8.624310,-11.519264,-30.856258,-32.514439,-50.491409,-30.370806
19427,8.696778,15.761197,21.532644,-65.125832,-51.503235,-59.551987,4.164798,15.069073,-28.311832,8.514996,-51.253830,16.239492,-18.043514,3.355330,-40.173748,-23.412256,-38.581284,-35.375664,-57.573151,-46.846222
19428,2.448001,7.151957,5.649827,-3.695565,-6.582603,-2.530823,1.253296,5.311233,-9.669888,6.486143,-8.986037,6.017430,-3.428592,2.726583,-4.636242,-8.341380,-9.649304,-4.928029,-5.482189,-1.434256
19429,10.985883,10.414428,13.659563,-14.063864,-19.012817,-36.729515,6.744277,10.182863,-14.065169,6.892967,-16.966274,7.804955,3.473657,3.786479,-13.157342,-10.071560,-21.367901,-11.526285,-19.210907,-14.197336


In [104]:
df.shape

(19430, 20)

In [139]:
#saving the third layer output for PCA analysis
df.to_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/20nodes_output_norm_leakyrelu.csv")

In [163]:
'''
#tSNE analysis on 20-nodes output
tsne=TSNE(n_components=2, n_iter=300, verbose=1)
embedding = tsne.fit_transform(df)

vis_x = embedding[:, 0]
vis_y = embedding[:, 1]

plt.scatter(vis_x, vis_y)
plt.savefig('20nodes_tSNE_leakyrelu.pdf',bbox_inches = 'tight')
'''

In [ ]:
'''
#contrastive PCA
foreground_data = pd.read_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/training_leaky_20nodes.csv",
                        index_col = 0)
foreground_data.shape

background_data = pd.read_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/testing_leaky_20nodes.csv",
                        index_col = 0)
background_data.shape


x = np.array(["1", "0"])
label = np.repeat(x, [287, 717], axis=0)

mdl = CPCA()
projected_data = mdl.fit_transform(foreground_data.values, background_data.values,plot = True, gui = True)
'''

In [105]:
import pickle as pkl 
import json 
from keras.utils import plot_model

In [106]:
weight_1 = Model1.get_weights()[0]
weight_2 = Model2.get_weights()[0]
weight_3 = Model3.get_weights()[0]

In [107]:
weight_1.shape

(359, 100)

In [108]:
weight_2.shape

(100, 50)

In [109]:
weight_3.shape

(50, 20)

In [71]:
Model1.get_weights()[1].shape

(100,)

In [63]:
'''
with open("weight_1",'wb') as f:
    pkl.dump(weight_1,f)
with open("weight_2",'wb') as f:
    pkl.dump(weight_2,f)
with open("weight_3",'wb') as f:
    pkl.dump(weight_3,f)
'''

In [110]:
# initializer for pre-trained weights; define functions for weight
def my_init1(shape, dtype = None):
    return weight_1

def my_init2(shape,dtype = None):
    return weight_2

def my_init3(shape,dtype = None):
    return weight_3


In [131]:
mlp = Sequential()
mlp.add(Dropout(0,input_shape=(359,)))
mlp.add(Dense(100,kernel_initializer = my_init1,activity_regularizer=regularizers.l1(10e-7)))
mlp.add(LeakyReLU(alpha = 0.3))
#mlp.add(Dropout(0.2))
mlp.add(Dense(50,kernel_initializer = my_init2,activity_regularizer=regularizers.l1(10e-7)))
#mlp.add(LeakyReLU(alpha = 0.3))
mlp.add(Dropout(0.2))
mlp.add(Dense(20,kernel_initializer = my_init3,activity_regularizer=regularizers.l1(10e-7)))
#mlp.add(LeakyReLU(alpha = 0.3))
mlp.add(Dense(1))
#mlp.add(LeakyReLU(alpha = 0.3))
mlp.compile(loss = 'binary_crossentropy',
		optimizer = 'adam',
		metrics = ['accuracy'])

In [132]:
X_train = pd.read_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/training_genes_norm.csv", 
                      index_col = 0)
X_train.shape
X_train.head()

,E001h3k4me3,E002h3k4me3,E003h3k4me3,E004h3k4me3,E005h3k4me3,E006h3k4me3,E007h3k4me3,E008h3k4me3,E009h3k4me3,E010h3k4me3,...,E056tensor,E059tensor,E089tensor,E090tensor,E094tensor,E097tensor,E098tensor,E100tensor,E109tensor,epit.entropy
ENSG00000004866,0.296625,0.105981,0.249337,0.184611,0.007758,-0.103241,-0.254451,-0.153236,0.567978,2.079856,...,0.525170,1.154039,0.094809,0.718738,0.699267,0.061643,1.580294,2.929246,0.954994,0.658541
ENSG00000005339,0.664484,0.469639,0.848853,1.325303,0.711186,0.610536,-0.522232,0.906287,0.822634,1.136299,...,-0.499322,0.003369,0.400658,-0.011319,0.565993,0.335474,0.384709,-0.146969,-0.043804,0.821583
ENSG00000005513,0.204398,1.045012,0.147311,-0.064777,1.585373,2.315699,-0.443473,0.536371,1.998647,0.870081,...,0.411337,0.642630,0.706508,0.171195,0.832542,0.198559,-0.158738,-0.293455,-0.614546,0.555202
ENSG00000006704,-0.035464,0.227201,0.362198,0.396463,1.144966,1.109917,1.588511,1.286247,0.893187,0.437056,...,0.069840,-0.252335,0.298708,0.079938,-0.233652,-0.075272,-0.267428,-0.439942,-0.186490,1.063549
ENSG00000007168,0.896981,0.367413,0.612749,1.201246,0.086256,0.387711,0.454905,0.145055,1.162038,1.392407,...,0.752834,0.898335,0.910407,0.536223,0.965816,1.156964,0.710778,0.878436,0.812308,0.392160


In [133]:
label = pd.read_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/training_genes_label.csv",
                   index_col = 0)
label.shape
label.head()

,label
1,1
2,1
3,1
4,1
5,1


In [134]:
training_vld = pd.read_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/training_genes_validation.csv",
                          index_col = 0)
training_label_vld = pd.read_csv(
                    "/Users/siying/Dropbox/Episcore_working/DeepLearning/data/training_genes_validation_label.csv",
                                index_col = 0)

In [135]:
training_vld.shape

(300, 359)

In [136]:
training_label_vld.shape

(300, 1)

In [138]:
mlp.fit(X_train.values,label.values,epochs = 50,batch_size = 64, 
        validation_data=(training_vld.values, training_label_vld.values))

Train on 1004 samples, validate on 300 samples
Epoch 1/50
1004/1004 [==============================] - 0s 82us/step - loss: 3.7572 - acc: 0.0110 - val_loss: 4.7477 - val_acc: 0.0200
Epoch 2/50
1004/1004 [==============================] - 0s 60us/step - loss: 5.2553 - acc: 0.0129 - val_loss: 5.3371 - val_acc: 0.0067
Epoch 3/50
1004/1004 [==============================] - 0s 61us/step - loss: 5.2374 - acc: 0.0070 - val_loss: 5.2855 - val_acc: 0.0067
Epoch 4/50
1004/1004 [==============================] - 0s 66us/step - loss: 5.2866 - acc: 0.0120 - val_loss: 5.0208 - val_acc: 0.0100
Epoch 5/50
1004/1004 [==============================] - 0s 78us/step - loss: 5.1245 - acc: 0.0139 - val_loss: 4.8497 - val_acc: 0.0167
Epoch 6/50
1004/1004 [==============================] - 0s 74us/step - loss: 4.9979 - acc: 0.0090 - val_loss: 4.8468 - val_acc: 0.0133
Epoch 7/50
1004/1004 [==============================] - 0s 59us/step - loss: 4.8062 - acc: 0.0060 - val_loss: 4.7173 - val_acc: 0.0100
Epoch 8/

In [155]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import roc_curve, auc 
model = linear_model.LogisticRegression()
model.fit(X_train.values, label.values)
predict_train_Y = model.predict(X_train.values)
print (X_train.shape, label.shape, predict_train_Y.shape)
label = label.reshape([1004])
train_acc = 1.0 * np.sum(predict_train_Y == label) / label.shape[0]
print (train_acc)
#predict_test_Y = model.predict(self.test_X)
#test_acc = 1.0 * np.sum(predict_test_Y == self.test_Y) /self.test_Y.size
#print ‘acc of train and test:’, train_acc, test_acc


AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [124]:
test = pd.read_csv("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/testing_genes_norm.csv", index_col = 0)

In [125]:
supervised_out = mlp.predict(test.values)

In [126]:
supervised_out.shape

(18426, 1)

In [128]:
supervised_out.to_cvs("/Users/siying/Dropbox/Episcore_working/DeepLearning/data/supervised_prediction.csv")

AttributeError: 'numpy.ndarray' object has no attribute 'head'